In [14]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
from utils.fetch_docs import fetch_documents
from langchain_google_genai import ChatGoogleGenerativeAI
from code_reflection_agent import final_agent





# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the tool AgentInstruction"""

In [15]:
class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")

class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)

def requirement_analysis_node(state: AgentBuilderState):
    
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0:
        return {"messages": [response]}
    
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    
    return {"messages": [response], "agent_instructions": agent_instructions}


def route_state(state: AgentBuilderState):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "agent_kernel_builder"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "requirement_analysis"


In [16]:

AGENT_KERNEL_PROMPT = PromptTemplate.from_template(
    """
        Task Overview:
        Design a langgraph StateGraph object implementing the the best architecture for the given set of requirements, tailored to fulfill the user requirements defined below

        <Requirements>
        Objectives: {objective}
        usecases: {responsibilities}
        examples: {examples}
        </Requirements>

        
        Expected Output:
        Your task is to produce a compiled StateGraph object.

        Guidelines for Code Generation:
        - Accuracy: Avoid hallucinations or speculative assumptions when writing code. Refer exclusively to the provided documentation.
        - Understanding: Thoroughly comprehend the architecture and examples of code.
        - Customization: Generate code tailored specifically to meet the user requirements.

    """)

def agent_kernel_builder(state: AgentBuilderState):
    """Build the agent kernel using the best architecture."""
    agent_instructions : AgentInstructions = state["agent_instructions"]
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    # agent_architecture_report.name
    #agent_architecture_report.highlights
    #agent_architecture_report.justification
    #agent_architecture_report.tailored_design
    print("reached here")
    response =  llm.invoke([HumanMessage(content=AGENT_KERNEL_PROMPT.format(
        objective=agent_instructions.objective,
        responsibilities=agent_instructions.usecases,
        examples = agent_instructions.examples,
        # langgraph_glossary=fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the generated agent kernel as the output
    return {
        "messages": [AIMessage(content="Generated agent kernel code!")],
        "python_code": response.content,
    }

In [17]:

CODE_TO_JSON_PROMPT = PromptTemplate.from_template("""
You are tasked with converting the following stategraph comPilation code into a JSON. 

Contextual documents for understanding code:
{documents}

The Input code is as follows:
{code_snippet}

OUTPUT: Explaination and JSON. Do not include any code blocks. Seperate the JSON and explaination blocks and ensure that there is an explaination for each line of JSON produced but keep the blocks seperated.
Each Output JSON will have a nodes sections containing all the nodes and an edges section

Please follow:
1. Produce the explaination first and then the JSON after it. DO not produce the JSON first. 
2. For any conditional edges, please include all the nodes that the source of a conditional edge can reach as part of the explaination.
3. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
4. A source can have any number of targets. Please write the explaination for each source node to target node edge
5. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
6. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
7. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
8. Every node should be a part of atleast one edge, Please ensure this is followed
9. Attach the code snippet for each node aswell that. Please extract it from the Input code


Example output JSON for a node:
    "code_node":{{
        "schema_info": /"/"/"CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str/"/"/",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code. This node returns command object",
    "function_name": "code_step"
    }}

Example output JSON for an edge:
edge:{{ source: "abc", target: "cde", routing_condition: "if abc made a tool call then go to cde", "conditional": true}}
edge:{{ source: "abc", target: "xyz", routing_condition: "if abc made an interupt to a human then go to xyz", "conditional": true}}
edge:{{ source: "xyz", target: "_END_", routing_condition: "no nodes to go after xyz, we have our final output for this path", "conditional": false}}s         
""")
def code_to_json_node(state: AgentBuilderState):
    """Convert the generated code to JSON."""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=CODE_TO_JSON_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated JSON code!")],
        "json_code": json_code_ouptut.content,
    }



In [18]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying and updating the provided JSON that represents the nodes and edges of the langgraph to ensure it is correct with respect to the input code.

Contextual Documents for Understanding Code:
{documents}

Input Code:
<Input code>{code_snippet}</Input code>
JSON:
<JSON>{node_json}</JSON>

OUTPUT: JSON
Your task is divided into two parts:
- Validation: Verify that the JSON adheres to the rules outlined below.
- Correction and Augmentation: If the JSON is incorrect or incomplete, update it with a clear justification for every change, and include the code snippet for each node extracted from the input code.

Rules for Validation and Update:
- Conditional Edges:- An edge can be marked as conditional: true only if its source acts as a source for multiple edges (i.e., at least two targets).
- If the source does not meet this condition, then it cannot have conditional edges.

- Edge Targets:- Each source can have any number of targets. This flexibility must be maintained.

- Start and End Nodes:- The JSON must begin with the __START__ node and conclude with the __END__ node, with correct edges to and from them. Edges into the END node can also be conditional if they meet the above mentioned conditions      

- Node Structure:- Each node entry in the nodes section of the JSON must include the following non-empty fields:- schema_info
- id
- schema_info
- input_schema
- output_schema
- description
- function_name


- Edge Structure:- Each edge entry in the edges section of the JSON must include the following non-empty fields:- source
- target
- routing_conditions
- conditional


- Node-Edge Relationship:- Every node must be part of at least one edge. Ensure this relationship is consistently followed.

- Node Code Snippets:- Attach a code field to every node in the JSON, extracted directly from the input code.

- Schema Requirements:- The final JSON must conform to the schema provided below:


Schema for JSON. Ensure the following schema is followed. No field should be missing for any node or edge:
- Node Example:

{{
  "code_node": {{
    "id" : "<id of the node, similar to name>"
    "schema_info": "<define the class structure of the state of this node. Also provide a name>",
    "input_schema": "<input state object name>",
    "output_schema": "<output state object name>",
    "description": "<description>",
    "function_name": "<function_name>",
    "code": "<python_code>"
  }}
}}


- Edge Example:

{{
  "edge": {{
    "source": "<source_node>",
    "target": "<target_node>",
    "routing_condition": "<routing_condition>",
    "conditional": true/false
  }}
}}


                                            
Key Instructions:
- Do not update any pre-existing field of the JSON unless you have an extremely strong justification for doing so.
- Clearly document the reasoning behind any additions, updates, or modifications to the JSON. Justifications should draw inspiration from the contextual documents mentioned earlier.
- Ensure conditional edges strictly adhere to the rules outlined above.
- Input_Schema and output_schemas can only have value None in JSON for START and END nodes. Please follow this without fail
- Include a code field for each code_node entry, with the exact code that corresponds to the node in the input code.


""")

def json_better_node(state: AgentBuilderState):
    """Add code to the json flow"""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=JSON_CODE_COMBINE_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        node_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated updated JSON code!")],
        "json_code": json_code_ouptut.content,
    }



In [19]:
CODE_GEN_PROMPT = PromptTemplate.from_template( """
You are an expert Python programmer specializing in LangGraph and AI agent development. Your primary task is to generate compilable Python code for a LangGraph state graph based on a JSON description that will be provided. You must carefully analyze the JSON to determine node functionalities, state management, and graph connectivity.

**Input:**
You will receive a JSON object with two main keys:
1.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\n type: TypedDict\n fields:\n - name: input\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. This might be a complete implementation or a simplified example.

2.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.

**Output Requirements:**
Generate a single, self-contained, and compilable Python script that performs the following:

1.  **Imports:** Include all necessary Python libraries (e.g., `typing` for `TypedDict`, `Optional`, `Dict`, `Any`; `langgraph.graph.StatefulGraph`, `langgraph.graph.START`, `langgraph.graph.END`; `langgraph.checkpoint.memory.InMemoryCheckpointer`; potentially `langchain_openai`, `langchain_core.pydantic_v1` for Pydantic models if LLM with structured output is needed; `re` if used in provided code snippets).

2.  **State Definition (`GraphState`):**
    * Parse the `schema_info` string from one of the node descriptions (assume they are consistent; if not, use the first one encountered).
    * Define a `GraphState` class using `typing.TypedDict` that accurately reflects the fields and their types as specified in `schema_info`. Pay close attention to `Optional` types and nested structures if any.

3.  **Node Implementation (Python Functions):**
    For each node defined in the `nodes` section of the JSON:
    * Create a Python function with the `function_name` provided. This function must accept the `GraphState` (the TypedDict you defined) as its sole argument and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation Strategy (CRITICAL):**
        * **Simple Algorithmic Logic:** If the `description` indicates a straightforward task and the provided `code` snippet is a complete, purely algorithmic Python function (e.g., string manipulation, regex-based parsing, simple dictionary operations) that fully implements this description without requiring advanced natural language understanding or complex inference, then directly use or adapt this provided `code` for the node's function.
        * **LLM Requirement:** If the `description` implies complex tasks such as:
            * Natural Language Understanding (NLU) (e.g., "robust intent classification", "understanding user queries")
            * Complex decision-making based on unstructured input
            * Text generation or summarization
            * Or if the `description` explicitly states "this function would typically use an LLM" or similar.
            Then, you MUST implement the node to utilize a Large Language Model (LLM).
            * If `code` is provided but is clearly a simplistic placeholder for what should be an LLM-driven task (e.g., basic keyword matching for intent classification when robust understanding is needed, as seen in the example's `classify_intent` node), you should replace or augment this simplistic code with an actual LLM integration.
            * When an LLM is required:
                * Include the necessary import for a common LLM client (e.g., `from langchain_openai import ChatOpenAI`).
                * In the node function, you can either instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo") # TODO: Replace with specific model and API key if needed`) or provide clear comment placeholders (e.g., `# Initialize LLM here (e.g., llm = ChatOpenAI(...))` followed by `# response = llm.invoke(...) # Adapt prompt and parsing based on node's task`).
                * The LLM should be prompted (even if schematically) to perform the task outlined in the node's `description`.
        * **Structured Output from LLM:** If the node's `description` or its purpose (e.g., extracting specific entities, parsing data into a predefined schema beyond simple `GraphState` fields) suggests that an LLM needs to produce a structured output (e.g., JSON, specific Pydantic model):
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tools for LLM:** If the `description` implies the node needs to interact with external systems, call specific APIs, or choose from a set of defined capabilities (e.g., "search customer database," "fetch external data," "invoke a specific service"):
            * Define appropriate LangChain tools (e.g., using `@tool` from `langchain_core.tools`).
            * If implementing an LLM call, bind these tools to the LLM. The LLM's role would be to decide which tool(s) to call based on the current state and input. For this generation task, you might define placeholder tools if the specifics are not in the JSON.

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

**Important Considerations:**
* The generated Python code must be complete, runnable, and accurately reflect the graph structure and logic described in the JSON.
* Ensure all node functions correctly update and return the relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Here is the JSON input:
<json>
{code_json}
</json>

Please generate the Python code now:
""")

def code_node(state: AgentBuilderState):
    """Produce the final python code"""
    code_ouptut = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        code_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_ouptut.content,
    }


In [ ]:
CODE_GEN_PROMPT = PromptTemplate.from_template( """
You are an expert Python programmer specializing in LangGraph and AI agent development. Your primary task is to generate compilable, logical, and complete Python code for a LangGraph state graph based on a JSON description. You must prioritize LLM-based implementations for relevant tasks and consider advanced graph architectures.

**Input:**
You will receive a JSON object with two main keys:
1.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\n type: TypedDict\n fields:\n - name: input\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy and overall architecture.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. **Treat this `code` primarily as an illustration or a very basic version. Prioritize LLM-based solutions if the `description` suggests a more robust approach is needed.**

2.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.

---
**Phase 1: Graph Architecture Analysis & Strategy**

Before generating any code, analyze the overall workflow implied by the `nodes` and `edges` in the JSON.
1.  **Identify Potential Architectures:** Consider if the described system aligns with or would benefit from known advanced LangGraph architectures such as:
    * **Plan and Execute**: Does the graph imply a planning step (e.g., breaking down a complex task) followed by the execution of those plans by one or more action nodes?
    * **Agent Supervisor / Hierarchical Agent Teams**: Do the nodes and their conditional routing suggest a supervisory agent dispatching tasks to specialized worker agents, or a hierarchy of agents making decisions and delegating?
    * **Multi-Agent Collaboration (e.g., Swarm Architecture)**: Does the problem benefit from multiple agents working in parallel or collaboratively, perhaps sharing insights or contributing to a common goal?
    * **Reflection / Self-Correction (e.g., Self-Discover frameworks)**: Are there indications of iterative refinement, where results are evaluated and the process is adjusted?
    * **Human in the Loop (HITL)**: Does the `description` of any node, or the overall process, imply a need for human review, approval, correction, or explicit input at specific stages (e.g., before executing a critical action, when confidence is low, or for subjective assessments)?

2.  **Architectural Decision:**
    * If you determine that one or more of these architectures are strongly applicable and would create a more robust or intelligent system based on the JSON's intent, choose to implement it.
    * This may require you to define additional coordinating nodes or logic (e.g., a dedicated 'planner' LLM-agent, a 'supervisor' LLM-agent that routes tasks) that are not explicitly listed as individual nodes in the input JSON but are integral to the chosen architecture.
    * If no specific advanced architecture seems directly applicable or sufficiently justified by the input JSON, proceed with a standard stateful graph construction based on the explicit nodes and edges.

3.  **Initial Comment:** At the very beginning of your generated Python script, include a comment block stating:
    * Which LangGraph architecture(s) (if any) you've identified and chosen to implement, with a brief justification based on your interpretation of the input JSON.
    * If you are proceeding with a standard graph, mention that.

---
**Phase 2: Python Code Generation**

Generate a single, self-contained, and compilable Python script that implements your chosen strategy.

1.  **Imports:** Include all necessary Python libraries (e.g., `typing`, `langgraph.graph`, `langgraph.checkpoint.memory`, LLM client libraries like `langchain_openai`, `langchain_google_genai`, `langchain_core.pydantic_v1`, `langchain_core.tools`, `re`).

2.  **State Definition (`GraphState`):**
    * Parse the `schema_info` string to define a `GraphState` class using `typing.TypedDict`.

3.  **Node Implementation (Python Functions):**
    For each conceptual node in your chosen architecture (these may map directly to JSON nodes or be new architectural nodes you define):
    * Create a Python function. This function must accept the `GraphState` and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation (Prioritize LLM, No Mock Data):**
        * **Default to LLM-Based Solutions:** Your default stance should be to implement an **LLM-based solution** if the node's `description` (from JSON or your architectural design) suggests tasks like:
            * Natural Language Understanding (NLU)
            * Complex classification or routing
            * Content generation or summarization
            * Tool selection and usage
            * Planning or complex decision-making.
            * Any task where an LLM would provide more robust, flexible, or intelligent behavior than simple hardcoded logic.
        * **Handling Provided `code`:** If `code` is present in the JSON for a node, treat it as a **low-priority hint or a simplistic example**. Do **not** simply copy it if an LLM approach is more appropriate for the described task.
        * **Algorithmic Logic (Use Sparingly):** Only use purely algorithmic Python code (like from the `code` attribute or written new) if the node's task is genuinely simple, deterministic (e.g., basic data formatting, fixed calculation), *and* an LLM would offer no significant benefit for that specific, narrow function.
        * **Functional LLM Calls:** When an LLM is used, instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo")` or `llm = ChatGoogleGenerativeAI(model="gemini-pro")`) and include a **functional, descriptive prompt** relevant to the node's task. Ensure the code for the LLM call is complete and not just a comment. Add a `TODO` comment for the user to specify API keys and potentially refine the model/prompt.
        * **No Mock Data:** Generated functions must be logical and aim for completeness. **Avoid using mock data or overly simplistic placeholder logic** where an LLM or a proper algorithmic implementation is expected.
        * **Structured Output & Tools:** If the task implies structured output from an LLM or the use of tools, define necessary Pydantic models and/or LangChain tools, and integrate them with the LLM call.
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tool Definition and Usage:** If a node's `description` (or your architectural design) implies the LLM within that node needs to interact with external systems, perform specific actions, or fetch data (e.g., "search customer database," "get weather update"):
                * Define these capabilities as discrete LangChain tools using the `@tool` decorator (e.g., `from langchain_core.tools import tool`).
                * **Crucially, each tool's internal Python function should be self-contained and directly perform its advertised action** (e.g., make a specific API call to an external service, run a local script, perform a calculation, retrieve data algorithmically). **Avoid embedding a *new, separate general-purpose LLM call within the tool's own implementation logic* unless the tool's explicit and documented purpose is to be a specialized, self-contained sub-agent (which is an advanced case).** The primary LLM within the graph node is responsible for *deciding to call* the tool and for interpreting its output.
                * Bind these well-defined tools to the LLM instance operating within that graph node. The node's LLM will then intelligently decide when to call a tool and with what inputs.
        * **Human in the Loop Nodes:** If you've designed a HITL step as a dedicated node, its function might primarily format data for human review and then process the subsequent human input (which would be added to the state, potentially by an external mechanism or a subsequent node). The graph might pause using an interruption mechanism tied to this node.
        * **State Coherence:** Ensure variable assignments and updates within node functions are coherent with the `GraphState` definition and how state is managed in LangGraph.

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

---
**Phase 3: Required Keys/Credentials Identification**

After generating the complete Python script, add a separate section at the end of your response, clearly titled:
`## Required Keys and Credentials`

In this section, list all environment variables or API keys a user would need to set for the generated code to execute successfully (e.g., `OPENAI_API_KEY`, `GOOGLE_API_KEY`, tool-specific keys). If no external keys are needed, state that.

---
**Important Considerations (General):**
* The primary goal is **compilable, logical, and functionally plausible Python code** that intelligently interprets the JSON input.
* Focus on creating a system that leverages LLMs effectively for tasks suited to them.
* Ensure node functions correctly update and return relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Here is the JSON input:
<json>
{code_json}
</json>

Please generate the Python code and the list of required keys now:
""")

def code_node(state: AgentBuilderState):
    """Produce the final python code"""
    code_ouptut = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        code_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_ouptut.content,
    }


In [39]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): I need to create a worklow with the objective of managing my social media.  It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post,  suggest me content I should be posting, make content for me if I give it a description * Identifying social media trends in sports. *   Finding relevant people for sponsoring specific posts. *   Suggesting content to post. *   Generating content based on a description.  Examples: Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should post a highlight reel
================================== Ai Mes

2025-06-01 02:34:30,252 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated JSON code!


2025-06-01 02:35:23,730 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated updated JSON code!
================================== Ai Message ==================================

Generated final python code!
User (q/Q to quit): q
AI: Byebye


In [ ]:
import json
dict_tool_link = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_link_json.json") )
dict_tool_doc = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_doc_json.json") )
def lowercase_keys(input_dict):
    """
    Returns a new dictionary with all keys converted to lowercase.
    """
    return {k.lower(): v for k, v in input_dict.items()}

dict_tool_link = lowercase_keys(dict_tool_link)
dict_tool_doc = lowercase_keys(dict_tool_doc)

Initial_prompt = """You are an expert python developer. You will be given a description of a python function. 

You job is to estimate and extract the following information:

- What exactly does this python do. What is the detailed objective of the function. Please write 1-5 lines
- Suggest or extract the name of the the function
- What would be the inputs/arguements required into this function to make it work. Please all mentioned the type of each input
- WHat would be output produced by this input. Please mention the output type 

Here is the description of the function you need to create:
<description>
{desc}
</description>
"""



class FunctionInstructions(BaseModel):
    """Instructions for defining a python function"""
    objective: str = Field(description= "what does this pythion function do")
    name: str = Field(description="name of the python function")
    input : List[str] = Field(description= "what would be the input arguements to this function along with the types")
    output: List[str] = Field(description="what would be the output/return attributes for the function along with the types")
    name_toolkit: str = Field(description="what would be the toolkit/ code SDK that will be used")
    code: str = Field(description="the final python code")
# Annotated[str, operator.add]

class CodebuilderState(BaseModel):
    """Instructions for defining a python function"""
    code: str = Field(description= "tailored code for the python function")


def functional_analysis_node(state: FunctionInstructions):
  print("functional_analysis_node")
  llm_with_structured_output = llm.with_structured_output(FunctionInstructions)
  functionalReport: FunctionInstructions = llm_with_structured_output.invoke(
      [HumanMessage(content=Initial_prompt.format(desc = state.objective))])
  return {  "messages": [AIMessage(content="Generated JSON code!")],
           "objective": functionalReport.objective,
           "name": functionalReport.name,
           "input": functionalReport.input,
           "output": functionalReport.output}

write_code_prompt = """
You are a skilled code generation assistant. Your task is to create executable code using the following information:
- SDK Documentation: The provided documentation outlines the functionalities and usage details of the SDK. Use this as the reference for constructing your code.
- Objective: A clear description of what the code is intended to achieve.
- Input: The expected input for the code (e.g., variables, parameters, data types).
- Output: The desired result or outcome of the code (e.g., format, type, or structure).
- SDK Name: The name of the SDK that must be used in the code.

Your goal is to generate executable code that:
- Adheres to the requirements outlined above.
- Follows standard coding practices and is optimized for readability and efficiency.
- Utilizes the specified SDK appropriately based on the documentation provided.
- Only return a self contained function
- Your output should only contain a code block containing the required function and nothing else. Please do no include any explainantions
- Write your code in python
- Please also provide which API keys will be required and define the API keys as part of the function
- Please also write the doc string for the python function
- Ensure that the function you produce is decorated with @tool. That means its defination should be preceeded by '@tool' in the line above

Here are some details about the python function you will be creating:
<objective>
{objective}
</objective>

<input schema>
{inputs}
</input schema>

<output schema>
{output}
</output schema>

<name of function>
{name}
</name of function>

Documentation for SDK that might be helpful:
<documentation>
{docs}
</documentation>

"""

Best_sdk_prompt = """
You are a highly specialized language model designed to assist in selecting the most suitable SDK for a given use case. You are provided with the following:
- A dictionary containing pairs of SDK names and their respective descriptions.
- Requirements for a piece of code, including the objective, input, and output.

Your task is to:
- Identify the SDK from the provided dictionary whose description best matches the given use case described in the code requirements.
- Also give preferences to SDKs that are generally more well known or are used more frequently in the industry (Use google tools for anything search related)
- Return only the name of the matching SDK without any additional text or formatting.
- Please ensure that the string you return is a valid key of the dictionary you get as input. PLEASE VERIFY THAT THE STRING YOU RETURN EXISTS AS A KEY IN THE INPUT DICTIONARY

Input Example:
Dictionary:
{{
"SDK_A": "[SDK_CC_ABC]Provides tools for web scraping and data extraction.",
"SDK_B": "Enables natural language processing for unstructured text.",
"SDK_C": "Facilitates the integration of payment gateways in applications."
}}
Code Requirements:
Objective: Extract data from multiple web pages.
Input: URLs of the web pages.
Output: Structured data in JSON format.

Expected Output:
SDK_A


Input :
<dictionary>
{dictionary}
</dictionary>

<objective>
{objective}
</objective>

<input schema>
{inputs}
</input schema>

<output schema>
{output}
</output schema>

<name of function>
{name}
</name of function>


"""


def sdk_production_node(state: FunctionInstructions):
    objective_agent: str = state.objective
    name: str = state.name
    input_args : List[str] = state.input
    output_args: List[str] = state.output
    response = llm.invoke([HumanMessage(content=Best_sdk_prompt.format(
          objective=objective_agent,
          inputs=input_args,
          output=output_args,
          name=name,
          dictionary = dict_tool_doc
    ))])
    code_snips = response
    return { "messages": [AIMessage(content="SDK identified")],
            "name_toolkit": response.content.lower()}

def code_production_node(state: FunctionInstructions):
    objective_agent: str = state.objective
    name: str = state.name
    input_args : List[str] = state.input
    output_args: List[str] = state.output
    toolkit: str = state.name_toolkit
    docs = fetch_documents(dict_tool_link[toolkit])
    response = llm.invoke([HumanMessage(content=write_code_prompt.format(
          objective=objective_agent,
          inputs=input_args,
          output=output_args,
          name=name,
          docs = docs,
    ))])
    code_snips = response
    return {"messages": [AIMessage(content="Generated code for tool")],
            "code": response.content}

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
C:\Users\anupa\AppData\Local\Temp\ipykernel_24008\3791049899.py:2: SyntaxWarning: invalid escape sequence '\A'
  dict_tool_link = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_link_json.json") )
C:\Users\anupa\AppData\Local\Temp\ipykernel_24008\3791049899.py:3: SyntaxWarning: invalid escape sequence '\A'
  dict_tool_doc = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_doc_json.json") )


In [27]:
from langgraph.checkpoint.memory import InMemorySaver
workflow = StateGraph(FunctionInstructions)
workflow.add_node("func_analysis", functional_analysis_node)
workflow.add_node("sdk_write", sdk_production_node)
workflow.add_node("code_write", code_production_node)
checkpointer = InMemorySaver()
workflow.add_edge("code_write", END)
workflow.add_edge("sdk_write","code_write")
workflow.add_edge("func_analysis","sdk_write")
workflow.add_edge(START, "func_analysis")
tool_infograph = workflow.compile(checkpointer=checkpointer)

In [87]:
class toolcollector(MessagesState):
    total_code: List[str]
    compiled_code: str
test_message = """
def get_weather(location: str):
    \"\"\"Call to get the current weather.\"\"\"
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."


@tool
def get_coolest_cities():
    \"\"\"Get a list of coolest cities\"\"\"
    return "nyc, sf"

tools = [get_weather, get_coolest_cities]

# Bind the model(llm) with tools
model_with_tools = ChatAnthropic(
    model="claude-3-haiku-20240307", temperature=0
).bind_tools(tools)

# Generate a tool node.
tool_node = ToolNode(tools)

# conditional edge
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END
"""
tool_desc_prompt = """
You are an AI assistant designed to analyze Python code. Your task is to identify all function definitions in the provided Python snippet that are decorated with @tool. You must return a dictionary where:
- The keys are the names of the identified functions.
- You only need to pick up a function if it is decorated with '@tool' or '@tool' just preceeds the function. Otherwise leave the function alone
- The values are descriptions of what each function is supposed to do. If a function contains a docstring, extract it as the description. If a docstring is missing, infer the function's purpose from its structure and comments.
- The output should just be a json. it should not include "```json" and "```" at the start or end of it. it should start with a "{{" and end with a "}}"
Example Input:
@tool
def calculate_area(length, width):
    "Calculates the area of a rectangle."
    return length * width

@tool
def greet(name):
    return f"Hello, {{name}}!"


Expected Output:
{{
    "calculate_area": "Calculates the area of a rectangle.",
    "greet": "Greets a user by name."
}}


Instructions:
- Identify functions that have the @tool decorator.
- Extract function names and descriptions (either from docstrings or inferred).
- Return the output as a structured JSON.
- Please only return a json object that can be converted into a json directly. DO NOT RETURN ANYTHING OTHER THAN A JSON

Python code:
<code>
{code}
</code>

"""

tool_compile_prompt = """
You are python code writing expert. You are given 2 snippets of code, your job is to combine them. 
The first snippet of code contains a compilable code with some functions compilable but empty. 
The second snippet of code contains the defination of those functions. 
Please fo through the second snippet of code, match the function in the first snippet and replace the functional definition written in the first snippet with one found in second snippet

Please only return compilable python code
Here are the code snippets:
<code_snippet1>
{complete_code}
</code_snippet1>
<code_snippet2>
{functions}
</code_snippet2>
"""

import uuid


def graph_map_step(state: toolcollector):
    # Extract nodes and edges from json_objects
    current_code = state['compiled_code']
    print(current_code)
    response_1 = llm.invoke([HumanMessage(content=tool_desc_prompt.format(code = current_code))])
    response_content =  response_1.content
    if(response_content[0]=='j'):
        json_objects = json.loads(response_content[8:-3])
    else:
        json_objects = json.loads(response_content)
    print(json_objects)
    uuid_str = uuid.uuid4()
    config = {"configurable": {"thread_id": str(uuid_str)}}
    send = []
    for key in json_objects:
        print(key + " " + json_objects[key])
        for output in tool_infograph.stream({"objective":key + " " + json_objects[key], "name": key, "input":[], "output": [], "name_toolkit": "", "code":""}, config, stream_mode="updates"):
            print(output)
        send.append(tool_infograph.get_state(config).values["code"])
    return {
        "messages": [AIMessage(content="Generated compiled code!")],
        "total_code": send,
        "compiled_code" : current_code
    }

def compile_code(state: toolcollector):
    tool_code_list = state['total_code']
    normal_code = state['compiled_code']
    full_tool_code = " ".join(tool_code_list)
    response = llm.invoke([HumanMessage(content=tool_compile_prompt.format(complete_code = normal_code, functions = full_tool_code))])
    return {
        "messages": response.content
    }




In [88]:
workflow1 = StateGraph(toolcollector)
# workflow1.add_node("tool_infograph", tool_infograph)
workflow1.add_node("graph_map_step", graph_map_step)
workflow1.add_node("compile_code", compile_code)

workflow1.add_edge(START, "graph_map_step")
workflow1.add_edge("graph_map_step","compile_code")
workflow1.add_edge("compile_code", END)
tool_compile = workflow1.compile()

In [89]:
def call_subgraph(state: AgentBuilderState):
    subgraph_output = tool_compile.invoke({"total_code": [], "compiled_code": state["python_code"]})  
    return {"python_code": subgraph_output["compiled_code"]}  

In [90]:
workflow = StateGraph(AgentBuilderState)
workflow.add_node("requirement_analysis", requirement_analysis_node)
workflow.add_node("agent_kernel_builder", agent_kernel_builder)
workflow.add_node("code_to_json", code_to_json_node)
workflow.add_node("json_update", json_better_node)
workflow.add_node("code_node",code_node)
# workflow.add_node("tool_compile",tool_compile)
workflow.add_node("node_1", call_subgraph)

@workflow.add_node
def add_tool_message(state: AgentBuilderState):
    
   
    return {
        "messages": [
            ToolMessage(
                content="Requirements generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

workflow.add_edge("node_1", END)
workflow.add_edge("code_node", "node_1")
workflow.add_edge("json_update", "code_node")
workflow.add_edge("code_to_json", "json_update")
workflow.add_edge("agent_kernel_builder", "code_to_json")
workflow.add_conditional_edges("requirement_analysis", route_state, ["agent_kernel_builder", "requirement_analysis", END])
workflow.add_edge(START, "requirement_analysis")
infograph = workflow.compile()

In [91]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        try:
            last_message.pretty_print()
        except:
            print(last_message)

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): I need to create a worklow with the objective of managing my social media.  It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post,  suggest me content I should be posting, make content for me if I give it a description * Identifying social media trends in sports. *   Finding relevant people for sponsoring specific posts. *   Suggesting content to post. *   Generating content based on a description.  Examples: Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should post a highlight reel
================================== Ai Mes

2025-06-01 15:41:41,934 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated JSON code!


2025-06-01 15:42:33,403 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated updated JSON code!
```python
import operator
from typing import TypedDict, List, Optional, Dict, Any

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemoryCheckpointer

# TODO: Choose your preferred LLM client.
# For OpenAI:
from langchain_openai import ChatOpenAI
# For Google GenAI:
# from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# --- Phase 1: Graph Architecture Analysis & Strategy ---
# This graph implements a Router Agent / Agent Supervisor architecture.
# The 'parse_input' node acts as a central router, analyzing the user's query
# to determine the primary t

...

2025-06-01 15:45:22,499 - INFO - HTTP Request: GET https://python.langchain.com/docs/integrations/tools/google_search "HTTP/1.1 308 Permanent Redirect"
2025-06-01 15:45:22,563 - INFO - HTTP Request: GET https://python.langchain.com/docs/integrations/tools/google_search/ "HTTP/1.1 200 OK"


{'code_write': {'code': ''}}
content_generator Generates a short piece of social media content based on a description.
functional_analysis_node
{'func_analysis': {'objective': 'This function generates a short piece of social media content. It takes a description as input and produces relevant social media content.', 'name': 'content_generator', 'input': ['description: str'], 'output': ['social_media_content: str']}}
{'sdk_write': {'name_toolkit': 'writer'}}


2025-06-01 15:50:04,609 - INFO - HTTP Request: GET https://python.langchain.com/docs/integrations/tools/writer "HTTP/1.1 308 Permanent Redirect"
2025-06-01 15:50:04,700 - INFO - HTTP Request: GET https://python.langchain.com/docs/integrations/tools/writer/ "HTTP/1.1 200 OK"


{'code_write': {'code': '```python\nimport os\nfrom langchain_core.tools import tool\nfrom langchain_writer.chat_models import ChatWriter\nfrom langchain_core.messages import HumanMessage\n\n@tool\ndef content_generator(description: str, writer_api_key: str) -> str:\n    """\n    Generates a short piece of social media content based on a given description.\n\n    Args:\n        description: A string describing the content for which social media content is to be generated.\n        writer_api_key: The API key for accessing the Writer AI Studio. This key is required to authenticate with the Writer API.\n\n    Returns:\n        A string containing the generated social media content.\n    """\n    # Set the Writer API key as an environment variable.\n    # The ChatWriter class expects the API key to be available in the environment.\n    os.environ["WRITER_API_KEY"] = writer_api_key\n\n    # Initialize the ChatWriter model.\n    chat = ChatWriter()\n\n    # Construct the prompt for generati

KeyError: 'messages'

In [ ]:
I need to create a worklow with the objective of managing my social media.
 It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post, 
suggest me content I should be posting, make content for me if I give it a description
* Identifying social media trends in sports.
*   Finding relevant people for sponsoring specific posts.
*   Suggesting content to post.
*   Generating content based on a description.

Examples:
Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending
If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc
If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should post a highlight reel

SyntaxError: invalid syntax (2812058957.py, line 1)

In [ ]:
```python
from langchain_community.tools.google_trends import GoogleTrendsQueryRun
from langchain_community.utilities.google_trends import GoogleTrendsAPIWrapper
from typing import List
from tool import tool
import os
from langchain_core.tools import tool
from langchain_writer.chat_models import ChatWriter
from langchain_core.messages import HumanMessage

@tool
def social_media_trend_search(sport: str, serpapi_api_key: str) -> List[str]:
    """
    Searches for current social media trends related to a specific sport using Google Trends.

    Args:
        sport (str): The name of the sport to search for trends.
        serpapi_api_key (str): Your SerpApi key for accessing Google Trends data.

    Returns:
        list[str]: A list of social media trends related to the specified sport.
                   This includes both rising and top related queries.
    """
    api_wrapper = GoogleTrendsAPIWrapper(serp_api_key=serpapi_api_key)
    google_trends_tool = GoogleTrendsQueryRun(api_wrapper=api_wrapper)

    query_result = google_trends_tool.run(sport)

    trends = []
    # Parse the output string to extract related queries
    # The output format is "Rising Related Queries: ...,Top Related Queries: ..."
    if "Rising Related Queries:" in query_result:
        rising_start = query_result.find("Rising Related Queries:") + len("Rising Related Queries:")
        top_start = query_result.find("Top Related Queries:")
        
        rising_queries_str = ""
        if top_start != -1:
            rising_queries_str = query_result[rising_start:top_start].strip()
        else:
            rising_queries_str = query_result[rising_start:].strip()

        if rising_queries_str:
            trends.extend([q.strip() for q in rising_queries_str.split(',') if q.strip()])

    if "Top Related Queries:" in query_result:
        top_start = query_result.find("Top Related Queries:") + len("Top Related Queries:")
        top_queries_str = query_result[top_start:].strip()
        if top_queries_str:
            trends.extend([q.strip() for q in top_queries_str.split(',') if q.strip()])
            
    # Remove duplicates and return
    return list(set(trends))

@tool
def content_generator(description: str, writer_api_key: str) -> str:
    """
    Generates a short piece of social media content based on a given description.

    Args:
        description: A string describing the content for which social media content is to be generated.
        writer_api_key: The API key for accessing the Writer AI Studio. This key is required to authenticate with the Writer API.

    Returns:
        A string containing the generated social media content.
    """
    # Set the Writer API key as an environment variable.
    # The ChatWriter class expects the API key to be available in the environment.
    os.environ["WRITER_API_KEY"] = writer_api_key

    # Initialize the ChatWriter model.
    chat = ChatWriter()

    # Construct the prompt for generating social media content.
    # The prompt instructs the AI to create engaging content suitable for social media.
    prompt = f"Generate a short, engaging social media post (e.g., for Twitter or Instagram) based on the following description: '{description}'"

    # Create a HumanMessage object with the constructed prompt.
    # This message will be sent to the ChatWriter model for processing.
    messages = [
        HumanMessage(content=prompt)
    ]

    # Invoke the ChatWriter model with the messages to get a response.
    response = chat.invoke(messages)

    # Extract the generated social media content from the response.
    # The 'content' attribute of the response object holds the AI's generated text.
    social_media_content = response.content

    return social_media_content
```